# DensityAD Quickstart

**DensityAD** (GradientCastDenseAD) uses advanced density-based pattern analysis to detect anomalies with severity classification.

**Best for:** Complex patterns, severity-based alerting, multi-feature analysis.

**How it works:**
1. Engineers features from historical data (rolling stats, z-scores, lags)
2. Applies intelligent density analysis to detect pattern-based outliers
3. Classifies severity (low, medium, high, critical)
4. Filters noise with contiguous anomaly requirements

**Important:** Historical data is required for context and feature engineering, but anomaly detection is performed only on the latest data point(s) in the time series you provide.

In [ ]:
import sys
sys.path.append('../..')

from gradientcast import GradientCastDenseAD
from utils.synthetic_data import generate_ad_payload_data
import matplotlib.pyplot as plt
import numpy as np

# Replace with your API key
GRADIENTCAST_API_KEY = "your-api-key-here"

dense_ad = GradientCastDenseAD(api_key=GRADIENTCAST_API_KEY)

---
## Basic Detection

In [ ]:
# Generate sample data with anomalies
# Earlier points serve as historical context, latest point(s) are evaluated
data = generate_ad_payload_data(n_points=100, inject_anomalies=True)

# Detect anomalies - uses history for context, evaluates latest point(s)
result = dense_ad.detect(data)

print(f"Alert Status: {result.alert_status}")
print(f"Alert Severity: {result.alert_severity}")
print(f"Total points in timeline: {len(result.timeline)}")
print(f"Confirmed anomalies: {len(result.anomalies)}")

---
## Understanding the Response

DensityAD provides rich anomaly metadata including severity levels.

In [ ]:
# Examine anomaly details
print("Anomaly Details:")
for point in result.anomalies[:5]:  # First 5
    mag = point.magnitude
    print(f"\n  Timestamp: {point.timestamp}")
    print(f"  Value: {point.value:,}")
    print(f"  Severity: {mag.severity.upper()}")
    print(f"  Anomaly Score: {mag.anomaly_score:.2f}")
    print(f"  Normalized Score: {mag.normalized_score:.1f}/100")
    print(f"  Z-Score (24h): {mag.zscore_24h:.2f}" if mag.zscore_24h else "  Z-Score: N/A")
    print(f"  Deviation: {mag.deviation_pct:+.1f}%")

In [ ]:
# Severity distribution
severity_counts = {}
for point in result.anomalies:
    sev = point.magnitude.severity
    severity_counts[sev] = severity_counts.get(sev, 0) + 1

print("Severity Distribution:")
for sev in ['low', 'medium', 'high', 'critical']:
    count = severity_counts.get(sev, 0)
    if count > 0:
        print(f"  {sev.capitalize()}: {count}")

---
## Visualizing Results

Color-coded by severity.

In [ ]:
# Extract data for plotting
values = [p.value for p in result.timeline]

# Color mapping for severity
severity_colors = {
    'none': 'blue',
    'low': 'yellow',
    'medium': 'orange',
    'high': 'red',
    'critical': 'darkred'
}

plt.figure(figsize=(14, 6))

# Plot time series
plt.plot(values, 'b-', alpha=0.5, lw=1, label='Values')

# Plot anomalies by severity
for sev in ['low', 'medium', 'high', 'critical']:
    sev_points = [(i, p.value) for i, p in enumerate(result.timeline) 
                  if p.confirmed_anomaly and p.magnitude.severity == sev]
    if sev_points:
        x, y = zip(*sev_points)
        plt.scatter(x, y, c=severity_colors[sev], s=100, zorder=5, 
                    edgecolors='black', linewidths=1, label=f'{sev.capitalize()}')

plt.xlabel('Time')
plt.ylabel('Value')
plt.title('DensityAD: Anomalies by Severity')
plt.legend(loc='upper right')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

---
## Raw vs Confirmed Anomalies

DensityAD distinguishes between initial detection (raw) and filtered results (confirmed).

In [ ]:
# Count raw vs confirmed
raw_count = sum(1 for p in result.timeline if p.raw_anomaly)
confirmed_count = sum(1 for p in result.timeline if p.confirmed_anomaly)

print(f"Raw anomalies (initial detection): {raw_count}")
print(f"Confirmed anomalies (after filtering): {confirmed_count}")
print(f"Filtered out: {raw_count - confirmed_count}")

# Filtering reasons:
# - Valley threshold: Values too low to be meaningful
# - Contiguous check: Single isolated points filtered out

In [ ]:
# Visualize raw vs confirmed
plt.figure(figsize=(14, 5))

plt.plot(values, 'b-', alpha=0.5, lw=1)

# Raw anomalies (light)
raw_only = [(i, p.value) for i, p in enumerate(result.timeline) 
            if p.raw_anomaly and not p.confirmed_anomaly]
if raw_only:
    x, y = zip(*raw_only)
    plt.scatter(x, y, c='lightblue', s=80, alpha=0.6, label='Raw (filtered out)')

# Confirmed anomalies
confirmed = [(i, p.value) for i, p in enumerate(result.timeline) if p.confirmed_anomaly]
if confirmed:
    x, y = zip(*confirmed)
    plt.scatter(x, y, c='red', s=100, zorder=5, edgecolors='darkred', 
                linewidths=1.5, label='Confirmed')

plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Raw vs Confirmed Anomalies')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

---
## Convert to DataFrame

In [ ]:
# Convert to pandas DataFrame
df = result.to_dataframe()
print("DataFrame columns:")
print(list(df.columns))
print()

# Show anomalies only
anomalies_df = df[df['confirmed_anomaly'] == True]
print(f"Anomalies DataFrame ({len(anomalies_df)} rows):")
anomalies_df[['timestamp', 'value', 'severity', 'normalized_score', 'deviation_pct']].head()

---
## Key Concepts

### Severity Levels

| Severity | Meaning | Action |
|----------|---------|--------|
| **Low** | Minor deviation | Log and monitor |
| **Medium** | Noticeable issue | Investigate |
| **High** | Significant problem | Alert on-call |
| **Critical** | Severe anomaly | Immediate action |

### Magnitude Metrics

| Metric | Description |
|--------|------------|
| `anomaly_score` | Raw density score (higher = more anomalous) |
| `normalized_score` | 0-100 scale for easy comparison |
| `zscore_24h` | Standard deviations from 24h mean |
| `deviation_pct` | Percentage deviation from expected |

### When to Use DensityAD

**Good for:**
- Complex, irregular patterns
- Need for severity classification
- Multi-feature analysis

**Consider PulseAD instead for:**
- Simple threshold-based detection
- Predictable trend deviations
- Lower latency requirements

**Next:** [Tuning Parameters](02_tuning.ipynb)